<a href="https://colab.research.google.com/github/kylewongAIHK/OOP-Java-UCSD-Coursera/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part I: Creating the Data Frame of the Greater Toronto Area

In [0]:
#Import relevant packages
import requests
import pandas as pd
import numpy as np
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [0]:
#Reading Web Site
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [0]:
# creating data from the website and clean up the table
df = pd.read_html(url, header=0)[0]
df = df[df.Borough != 'Not assigned']
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

In [70]:
#examining the dataframe
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [71]:
df.shape

(103, 3)

## Part 2a: Adding the latitude and the longitude coordinates of each neighborhood.

In [72]:
#Reading the co-ordinate data from http://cocl.us/Geospatial_dat
url="http://cocl.us/Geospatial_data"
df2=pd.read_csv(url)
df2.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [73]:
df2.columns = ['Postcode', 'Latitude', 'Longitude']
toronto = pd.merge(df,df2, on='Postcode')
toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part 3a: Creating the Map of the Greater Toronto Area

In [74]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [75]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Part 3b: Exploring Scarborough

In [76]:
scarborough_data = toronto[toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [77]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [78]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Part 3c: Let's exlore the second neighbourhood in our Scaborough Data Frame

In [79]:
scarborough_data.loc[1, 'Neighbourhood']

'Highland Creek, Rouge Hill, Port Union'

In [80]:
neighborhood_latitude = scarborough_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[1, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Highland Creek, Rouge Hill, Port Union are 43.7845351, -79.16049709999999.


### Part 3d: Now, let's get the top 100 venues that are in Rouge, Malvern within a radius of 500 meters.

In [81]:
CLIENT_ID = 'FM1TPJEOHSEVIIBEKB5P45DEV34XSOK4T0QQ5D0NPRMJY1UN' # your Foursquare ID
CLIENT_SECRET = 'IIIVFQTFUKRA5ZTFMZ3C4YAIEQG2XNSHMFSIWFUCW1YVKRGL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FM1TPJEOHSEVIIBEKB5P45DEV34XSOK4T0QQ5D0NPRMJY1UN
CLIENT_SECRET:IIIVFQTFUKRA5ZTFMZ3C4YAIEQG2XNSHMFSIWFUCW1YVKRGL


In [82]:
# type your answer here
LIMIT = 100
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude,
        neighborhood_longitude,
        radius,
        LIMIT)

url



'https://api.foursquare.com/v2/venues/explore?client_id=FM1TPJEOHSEVIIBEKB5P45DEV34XSOK4T0QQ5D0NPRMJY1UN&client_secret=IIIVFQTFUKRA5ZTFMZ3C4YAIEQG2XNSHMFSIWFUCW1YVKRGL&v=20180605&ll=43.7845351,-79.16049709999999&radius=500&limit=100'

In [0]:
results = requests.get(url).json()

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [85]:
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Royal Canadian Legion,Bar,43.782533,-79.163085
1,Scarborough Historical Society,History Museum,43.788755,-79.162438


In [86]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### This section of Scarborough is definitely not very interesting

### Part 3e: Scarborough Neighbourhood Exploration

Now let's explore whole Scarborough

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [89]:
print(scarborough_venues.shape)
scarborough_venues.head()

(88, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [90]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6


In [91]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 51 uniques categories.


### Part 3f: Analysing Each Neighbourhood of Scarborough

In [92]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [93]:
scarborough_onehot.shape

(88, 52)

### Part 3g: Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [94]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.200000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.200000,0.000000,0.20,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000
3,Cedarbrae,0.0,0.142857,0.0,0.142857,0.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.142857,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.0,0.222222,0.000000,0.0,0.000000,0.000000,0.222222,0.111111,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.0,0.000000,0.083333,0.0,0.083333,0.000000,0.000000,0.000000,0.00,0.000000,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.083333,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.083333,0.000000,0.000000,0.083333,0.166667,0.0,0.000000,0.000000,0.083333,0.00,0.000000,0.083333,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.5,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

### Let's print each neighborhood along with the top 5 most common venues

In [95]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0        Skating Rink   0.2
1      Breakfast Spot   0.2
2      Sandwich Place   0.2
3              Lounge   0.2
4  Chinese Restaurant   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               venue  freq
0               Park   0.5
1         Playground   0.5
2       Noodle House   0.0
3         Hobby Shop   0.0
4  Indian Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0    Hakka Restaurant  0.14
1     Thai Restaurant  0.14
2              Bakery  0.14
3                Bank  0.14
4  Athletics & Sports  0.14


----Clairlea, Golden Mile, Oakridge----
          venue  freq
0        Bakery  0.22
1      Bus Line  0.22
2  Intersection  0.11
3  Soccer Field  0.11
4   Bus Station  0.11

### Part 3h: Let's put that into a pandas dataframe

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)


neighborhoods_venues_sorted.rename(columns={'Neighborhood': 'Neighbourhood'}, inplace=True)
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Agincourt,Chinese Restaurant,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Café,Caribbean Restaurant,Grocery Store,Bus Station,Bus Line
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Vietnamese Restaurant,Caribbean Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Café,Grocery Store,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Caribbean Restaurant,Gym,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
3,Cedarbrae,Hakka Restaurant,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Athletics & Sports,Convenience Store,Grocery Store,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium,Bar,Coffee Shop,Gym,Auto Garage
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Intersection,Park,Bus Station,Vietnamese Restaurant,Coffee Shop,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Café,Chinese Restaurant,Caribbean Restaurant,Grocery Store


## Part 3i: Cluster Neighborhoods of Scarborough

In [98]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([1, 0, 1, 1, 1, 1, 4, 1, 1, 1, 3, 1, 1, 2, 0, 1], dtype=int32)

In [99]:
neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Agincourt,Chinese Restaurant,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Café,Caribbean Restaurant,Grocery Store,Bus Station,Bus Line
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Vietnamese Restaurant,Caribbean Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Café,Grocery Store,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Caribbean Restaurant,Gym,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
3,Cedarbrae,Hakka Restaurant,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Athletics & Sports,Convenience Store,Grocery Store,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium,Bar,Coffee Shop,Gym,Auto Garage
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Intersection,Park,Bus Station,Vietnamese Restaurant,Coffee Shop,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Café,Chinese Restaurant,Caribbean Restaurant,Grocery Store
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Chinese Restaurant,Italian Restaurant,Noodle House,Breakfast Spot,Pharmacy,Fried Chicken Joint,Bank,Shopping Mall,Thai Restaurant,Fast Food Restaurant,Discount Store,Electronics Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Vietnamese Restaurant,Caribbean Restaurant,Café
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Athletics & Sports,Auto Garage,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Gym,Caribbean Restaurant,Café,Bus Station,Bus Line
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Pet Store,Latin American Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Caribbean Restaurant,Grocery Store,Café,Bus Station,Bus Line,Bubble Tea Shop
8,"East Birchmount Park, Ionview, Kennedy Park",Chinese Restaurant,Hobby Shop,Discount Store,Department Store,Convenience Store,Coffee Shop,Gym,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,College Stadium,Vietnamese Restaurant,Caribbean Restaurant,Café,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
9,"Guildwood, Morningside, West Hill",Mexican Restaurant,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Vietnamese Restaurant,General Entertainment,Café,Bus Station


In [100]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scarborough_merged # check the last columns!



,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Caribbean Restaurant,Gym,Café,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot,Bar
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,History Museum,Bar,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant,Gym,Caribbean Restaurant,Café,Bus Station,Bus Line,Bubble Tea Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Mexican Restaurant,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Vietnamese Restaurant,General Entertainment,Café,Bus Station
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Caribbean Restaurant,Grocery Store,Café,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Athletics & Sports,Convenience Store,Grocery Store,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium,Bar,Coffee Shop,Gym,Auto Garage
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Health & Beauty Service,Playground,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Caribbean Restaurant,Grocery Store,Café,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,1.0,Chinese Restaurant,Hobby Shop,Discount Store,Department Store,Convenience Store,Coffee Shop,Gym,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,College Stadium,Vietnamese Restaurant,Caribbean Restaurant,Café,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,1.0,Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Intersection,Park,Bus Station,Vietnamese Restaurant,Coffee Shop,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Café,Chinese Restaurant,Caribbean Restaurant,Grocery Store
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,4.0,American Restaurant,Motel,Athletics & Sports,Auto Garage,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Gym,Caribbean Restaurant,Café,Bus Station,Bus Line
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1.0,General Entertainment,Skat

In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    if np.isnan(cluster): cluster = 0.0
    cluster =int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Part 3j: Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [102]:
#Cluster 0
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
5,Scarborough,0.0,Health & Beauty Service,Playground,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Caribbean Restaurant,Grocery Store,Café,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
14,Scarborough,0.0,Playground,Park,Vietnamese Restaurant,Caribbean Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Café,Grocery Store,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot


In [103]:
#Cluster 1
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,Scarborough,1.0,Mexican Restaurant,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Caribbean Restaurant,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Vietnamese Restaurant,General Entertainment,Café,Bus Station
3,Scarborough,1.0,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Caribbean Restaurant,Grocery Store,Café,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
4,Scarborough,1.0,Hakka Restaurant,Bank,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Athletics & Sports,Convenience Store,Grocery Store,General Entertainment,Gaming Cafe,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,College Stadium,Bar,Coffee Shop,Gym,Auto Garage
6,Scarborough,1.0,Chinese Restaurant,Hobby Shop,Discount Store,Department Store,Convenience Store,Coffee Shop,Gym,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,College Stadium,Vietnamese Restaurant,Caribbean Restaurant,Café,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
7,Scarborough,1.0,Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Intersection,Park,Bus Station,Vietnamese Restaurant,Coffee Shop,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Café,Chinese Restaurant,Caribbean Restaurant,Grocery Store
9,Scarborough,1.0,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,Coffee Shop,Caribbean Restaurant,Gym,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot
10,Scarborough,1.0,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Pet Store,Latin American Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Caribbean Restaurant,Grocery Store,Café,Bus Station,Bus Line,Bubble Tea Shop
11,Scarborough,1.0,Auto Garage,Shopping Mall,Sandwich Place,Bakery,Breakfast Spot,Vietnamese Restaurant,Coffee Shop,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Café,Chinese Restaurant,Caribbean Restaurant,Grocery Store,Bus Station
12,Scarborough,1.0,Chinese Restaurant,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Café,Caribbean Restaurant,Grocery Store,Bus Station,Bus Line
13,Scarborough,1.0,Pizza Place,Chinese Restaurant,Italian Restaurant,Noodle House,Breakfast Spot,Pharmacy,Fried Chicken Joint,Bank,Shopping Mall,Thai Restaurant,Fast Food Restaurant,Discount Store,Electronics Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Vietnamese Restaurant,Caribbean Restaurant,Café


In [104]:
#Cluster 2
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Caribbean Restaurant,Gym,Café,Bus Station,Bus Line,Bubble Tea Shop,Breakfast Spot,Bar


In [105]:
#Cluster 3
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,Scarborough,3.0,History Museum,Bar,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Chinese Restaurant,Gym,Caribbean Restaurant,Café,Bus Station,Bus Line,Bubble Tea Shop


In [106]:
#Cluster 4
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
8,Scarborough,4.0,American Restaurant,Motel,Athletics & Sports,Auto Garage,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store,College Stadium,Coffee Shop,Chinese Restaurant,Gym,Caribbean Restaurant,Café,Bus Station,Bus Line
